*UE Learning from User-generated Data, CP MMS, JKU Linz 2023*
# Exercise 5: Popularity bias evaluation

In this exercise we evaluate popularity bias of three different RecSys we already implemented. Note that here we utilize only one possible approach to evaluation of popularity bias from the user perspective. Refer to the slides and the recording for more details.

This time recomendations for each algorithm are already provided in three separate files (can be found on the main page).

Evaluating popularity bias through the mismatch (miscalibration) between popularity distributions over users' listening histories and their respective recommendations requires knowing how popular each item of the dataset is. Please use the new version of the lfm-tiny dataset where popularity categories (`popularity_bins`) from `0` (LowPop) to `2` (HighPop) have been already assigned. You will not need the test set for this exercise.

Make sure to rename the notebook according to the convention:

LUD23_ex05_k<font color='red'><Matr. Number\></font>_<font color='red'><Surname-Name\></font>.ipynb

for example:

LUD23_ex05_k000007_Bond_James.ipynb

## Implementation
In this exercise, as before, you are reqired to write a number of functions. Only implemented functions are graded. Insert your implementations into the templates provided. Please don't change the templates even if they are not pretty. Don't forget to test your implementation for correctness and efficiency. **Make sure to try your implementations on toy examples and sanity checks.**

Please **only use libraries already imported in the notebook**.

In [1]:
import numpy as np
import pandas as pd

from rec import inter_matr_implicit

## Task 1

Implement Kullback–Leibler divergence (KL) and Jensen–Shannon divergence (JSD).

#### in KL
Make sure to
* use the logarithm with base 2
* add `eps` to both `P` and `Q` distributions
* **normalize** (convert to probability) the distributions again after you have added `eps`

#### for JSD
* feel free to call your implementation of KL from it
* refer to the recording and slides for a recap

In [2]:
def KL(P: np.ndarray, Q: np.ndarray, eps: float = 1e-14):
    '''
    P - np.ndarray, probability distribution
    Q - np.ndarray, probability distribution
    eps - float, margin added to both distributions to avoid division by zero errors
    returns - float, divergence of distributions P,Q
    '''

    kl = None

    # TODO: YOUR IMPLEMENTATION
    
    # Add epsilon to avoid division by zero errors
    P = P + eps
    Q = Q + eps

    # Normalize the distributions
    P = P / np.sum(P)
    Q = Q / np.sum(Q)

    # Calculate KL divergence
    kl = np.sum(P * np.log2(P / Q))
    
    # Convert numpy to native python scalar [For Assertion Only]
    kl = kl.item()
    return kl


In [3]:
# Check if you handle zeros correctly (use "eps" parameter)
P = np.array([0.3, 0.6, 0.1])
Q = np.array([0.4, 0.6, 0.0])

assert type(KL(P, Q)) == float
assert np.isclose(KL(P, Q), 4.193995)

In [4]:
def JSD(P: np.ndarray, Q: np.ndarray, eps: float = 1e-14):
    '''
    P - np.ndarray, probability distribution
    Q - np.ndarray, probability distribution
    eps - float, margin added to both distributions to avoid division by zero errors, parameter for KL
    returns - float, divergence of distributions P,Q
    '''

    jsd = None

    # TODO: YOUR IMPLEMENTATION
    # Calculate average distribution
    M = 0.5 * (P + Q)

    # Calculate JSD
    jsd = 0.5 * (KL(P, M, eps) + KL(Q, M, eps))
    
    # Convert numpy to native python scalar [For Assertion Only]
    jsd = jsd
    return jsd

In [5]:
# Check if you handle zeros correctly (use "eps" parameter)
P = np.array([0.3, 0.6, 0.1])
Q = np.array([0.4, 0.6, 0.0])

assert type(JSD(P, Q)) == float
assert np.isclose(JSD(P, Q), 0.055170)

In [6]:
# KL divergence compares a target distribution to a reference distribution and is NOT symmetric
# JS divergence is symmetric -  you can exchange P and Q and get the same results
# the following asserts check this behaviour
P = np.array([0.3, 0.6, 0.1])
Q = np.array([0.2, 0.3, 0.5])

assert KL(P, Q) != KL(Q, P)
assert JSD(P, Q) == JSD(Q, P)

## Task 2

Evaluating algorithms for popularity bias.
First make sure all the data is loaded correctly.

In [7]:
def read(dataset, file):
    return pd.read_csv(dataset + '/' + dataset + '.' + file, sep='\t')


items = read("lfm-tiny", 'item')
users = read("lfm-tiny", 'user')
train_inters = read("lfm-tiny", 'inter_train')

train_inter_matrix = inter_matr_implicit(users=users, items=items, interactions=train_inters, dataset_name="lfm-tiny")

# Information about item popularity: 0 - LowPop, 2 - HighPop
item_pop_bins = items["popularity_bins"].to_numpy()
# Train set - users' listening history!
train_interaction_matrix = train_inter_matrix

In [8]:
config = {
    "item_pop_bins": item_pop_bins,
    "user_histories": train_interaction_matrix,
    "recommenders": {}  # here you can access the recommendations for every algorithm
}

recommendations = {"recommenders": {
    "SVD": {
        "recommendations": np.array([])
    },
    "ItemKNN": {
        "recommendations": np.array([])
    },
    "TopPop": {
        "recommendations": np.array([])
    },
}}

# Loading predictions
for recommender in recommendations["recommenders"].keys():
    recommendations["recommenders"][recommender]["recommendations"] = np.load(f"recommendations/{recommender}.npy")

config.update(recommendations)

Implement `evaluate_jsd` and then call it from `evaluate_algorithms`. Do not use any global variables.
For each user construct the two popularity distributions (from history and recommendations), normalize them, push through JSD and return the overall mean over users as the result.

In [9]:
def evaluate_jsd(recommendations, user_histories, item_pop_bins):
    '''
    recommendations - np.ndarray, 2d array containing indices of recommended items for every user
    user_histories - list of np.ndarray, containing the item indices of user interactions from the training set
    item_pop_bins - np.ndarray, 1d array that contains the popularity bin number of an item at the respective index
    returns - float, mean of jensen-shannon divergence over all users
    '''

    mean_jsd = None

    # TODO: YOUR IMPLEMENTATION
    num_users = len(recommendations)
    jsd_values = []

    for user_id in range(num_users):
        recommendation_indices = recommendations[user_id]
        history_indices = user_histories[user_id]
        
        # Construct popularity distributions from history and recommendations
        history_popularity = np.bincount(item_pop_bins[history_indices], minlength=len(item_pop_bins))
        recommendation_popularity = np.bincount(item_pop_bins[recommendation_indices], minlength=len(item_pop_bins))

        # Normalize the distributions
        history_popularity = history_popularity / np.sum(history_popularity)
        recommendation_popularity = recommendation_popularity / np.sum(recommendation_popularity)

        # Calculate JSD
        jsd = JSD(history_popularity, recommendation_popularity)
        jsd_values.append(jsd)

    # Calculate the mean JSD over all users
    mean_jsd = np.mean(jsd_values)
    return mean_jsd

In [10]:
user_histories = [np.array([5, 6]), np.array([7, 8, 9])]
item_pop_bins = np.array([0, 0, 0, 0, 0, 0, 0, 1, 2, 2])
recommendations = np.array([[0, 1, 8, 3, 4], [0, 1, 2, 3, 4]])

jsd = evaluate_jsd(recommendations, user_histories, item_pop_bins)

assert np.isclose(jsd, 0.554015)

`evaluate_algorithms` takes the config as an input and evaluates popularity bias of all algorithms listed in it.

In [11]:
def evaluate_algorithms(config):
    '''
    config - dictionary, provides the run config for the evaluation, use the already provided "config"
    returns - dictionary, result dictionary as provided within the function
    '''
    result = {
        "recommenders": {# jsd per algorithm stored here
            "SVD": {
                "jsd": 0
            },
            "ItemKNN": {
                "jsd": 0
            },
            "TopPop": {
                "jsd": 0
            }
        }
    }
    # TODO: YOUR IMPLEMENTATION
    user_histories = config["user_histories"]
    item_pop_bins = config["item_pop_bins"]
        
    for algorithm in result["recommenders"]:
        recommendations = config["recommenders"][algorithm]["recommendations"]
        mean_jsd = evaluate_jsd(recommendations, user_histories, item_pop_bins)

        result["recommenders"][algorithm]["jsd"] = mean_jsd
        
    return result

In [12]:
result = evaluate_algorithms(config)
assert isinstance(result["recommenders"]["SVD"]["jsd"], float)

In [13]:
# Investigate your results - which algorithm has the lowest bias/divergence?
print(result)

{'recommenders': {'SVD': {'jsd': 0.6828240626375526}, 'ItemKNN': {'jsd': 0.5168217834320056}, 'TopPop': {'jsd': 0.9771289695391738}}}
